<a href="https://colab.research.google.com/github/QuickLearner171998/CapsNet/blob/master/capsnet_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd gdrive
%cd 'My Drive'
%cd 'MY Projects'
%cd 'EEE lop'
%cd 'tensorflow_implementation'
!ls


/content/gdrive
/content/gdrive/My Drive
/content/gdrive/My Drive/MY Projects
/content/gdrive/My Drive/MY Projects/EEE lop
/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation
'capsnet tf.ipynb'   data     logdir_try8   tensorboard.ipynb   Try-10	 Try-4
 capsnet_tf.py	     logdir   results	    Try-1	        Try-3	 try-5


In [0]:
import os
import scipy
import numpy as np
import tensorflow as tf
# from tqdm import tqdm
tf.logging.set_verbosity(tf.logging.INFO)
import random
import skimage.io
import matplotlib.pyplot as plt
from skimage import transform



# Utils


Data Augmentation


In [0]:
def data_aug(images,labels,angel,resize_rate, populate):


  new_img = []
  new_label = []
  print("\nStarting Data Augmentation")
  for img,label in zip(images,labels):

    image = img
    #label1 = label.reshape(1,1)
    #flip = random.randint(0, 1)
    size = image.shape[0]

    sh = random.random()/2-0.25
    rotate_angel = random.random()/180*np.pi*angel
    # Create Afine transform
    afine_tf = transform.AffineTransform(shear=sh,rotation=rotate_angel)
    # Apply transform to image data
    image = transform.warp(image, inverse_map=afine_tf,mode='edge')
    
    new_img.append(image)
    new_label.append(label)
  
  print("\nFinished Augmentation")
  if(populate):

    final_trX = np.asarray(images + new_img)
    final_labels = np.asarray(labels + new_label)
    return final_trX, final_labels
  return (np.array(new_img)).astype('float32'), (np.array(labels,dtype='int32').astype('int32'))

In [0]:
def load_mnist_kannada(batch_size, is_training=True):
    if is_training:
        fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Kannada_MNIST/X_kannada_MNIST_train-idx3-ubyte')
        loaded = np.fromfile(file=fd, dtype=np.uint8)
        trainX = loaded[16:].reshape((60000, 28, 28, 1)).astype(np.float32)

        fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Kannada_MNIST/y_kannada_MNIST_train-idx1-ubyte')
        loaded = np.fromfile(file=fd, dtype=np.uint8)
        trainY = loaded[8:].reshape((60000)).astype(np.int32)

        trX = trainX[:50000] / 255.
        trY = trainY[:50000]

        trX, trY = data_aug(list(trX),list(trY),angel=5,resize_rate=0.9,populate=False)

        valX = trainX[50000:, ] / 255.
        valY = trainY[50000:]

        num_tr_batch = 50000 // batch_size
        num_val_batch = 10000 // batch_size

        return trX, trY, num_tr_batch, valX, valY, num_val_batch
    else:
        
        # test on 60K dataset
        #fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Kannada_MNIST/X_kannada_MNIST_test-idx3-ubyte')
        
        # test on DIG 10K 
        fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Dig_MNIST/X_dig_MNIST-idx3-ubyte.gz (Unzipped Files)/X_dig_MNIST-idx3-ubyte')
        
        loaded = np.fromfile(file=fd, dtype=np.uint8)
        n_test_img = ((len(loaded)-16))//(28*28)
        teX = loaded[16:].reshape((n_test_img, 28, 28, 1)).astype(np.float)
        
        # test on 60K
        #fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Kannada_MNIST/y_kannada_MNIST_test-idx1-ubyte')
        
        # test on 10K
        fd = open('/content/gdrive/My Drive/MY Projects/EEE lop/tensorflow_implementation/data/Kannada_MNIST_datataset_paper/Kannada_MNIST_Ubyte_gz/Dig_MNIST/y_dig_MNIST-idx1-ubyte.gz (Unzipped Files)/y_dig_MNIST-idx1-ubyte')
        
        loaded = np.fromfile(file=fd, dtype=np.uint8)
        teY = loaded[8:].reshape((n_test_img)).astype(np.int32)

        num_te_batch = n_test_img // batch_size
        return teX / 255., teY, num_te_batch




def load_data(batch_size, is_training=True, one_hot=False):
    return load_mnist_kannada(batch_size, is_training)
    

def get_batch_data(batch_size, num_threads):
    trX, trY, num_tr_batch, valX, valY, num_val_batch = load_mnist_kannada(batch_size, is_training=True)
    data_queues = tf.train.slice_input_producer([trX, trY])
    X, Y = tf.train.shuffle_batch(data_queues, num_threads=num_threads,
                                  batch_size=batch_size,
                                  capacity=batch_size * 64,
                                  min_after_dequeue=batch_size * 32,
                                  allow_smaller_final_batch=False)

    return(X, Y)


def save_images(imgs, size, path):
    '''
    Args:
        imgs: [batch_size, image_height, image_width]
        size: a list with tow int elements, [image_height, image_width]
        path: the path to save images
    '''
    imgs = (imgs + 1.) / 2  # inverse_transform
    return(scipy.misc.imsave(path, mergeImgs(imgs, size)))


def mergeImgs(images, size):
    h, w = images.shape[1], images.shape[2]
    imgs = np.zeros((h * size[0], w * size[1], 3))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        imgs[j * h:j * h + h, i * w:i * w + w, :] = image

    return imgs


# For version compatibility
def reduce_sum(input_tensor, axis=None, keepdims=False):
    try:
        return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
    except:
        return tf.reduce_sum(input_tensor, axis=axis, keep_dims=keepdims)


# For version compatibility
def softmax(logits, axis=None):
    try:
        return tf.nn.softmax(logits, axis=axis)
    except:
        return tf.nn.softmax(logits, dim=axis)


def get_shape(inputs, name=None):
    name = "shape" if name is None else name
    with tf.name_scope(name):
        static_shape = inputs.get_shape().as_list()
        dynamic_shape = tf.shape(inputs)
        shape = []
        for i, dim in enumerate(static_shape):
            dim = dim if dim is not None else dynamic_shape[i]
            shape.append(dim)
        return(shape)


# CapsLayer




In [0]:
"""
License: Apache-2.0
Author: Huadong Liao
E-mail: naturomics.liao@gmail.com
"""

epsilon = 1e-9


class CapsLayer(object):
    ''' Capsule layer.
    Args:
        input: A 4-D tensor.
        num_outputs: the number of capsule in this layer.
        vec_len: integer, the length of the output vector of a capsule.
        layer_type: string, one of 'FC' or "CONV", the type of this layer,
            fully connected or convolution, for the future expansion capability
        with_routing: boolean, this capsule is routing with the
                      lower-level layer capsule.

    Returns:
        A 4-D tensor.
    '''
    def __init__(self, num_outputs, vec_len, with_routing=True, layer_type='FC'):
        self.num_outputs = num_outputs
        self.vec_len = vec_len
        self.with_routing = with_routing
        self.layer_type = layer_type

    def __call__(self, input, kernel_size=None, stride=None):
        '''
        The parameters 'kernel_size' and 'stride' will be used while 'layer_type' equal 'CONV'
        '''
        if self.layer_type == 'CONV':
            self.kernel_size = kernel_size
            self.stride = stride

            if not self.with_routing:
                # the PrimaryCaps layer, a convolutional layer
                # input: [batch_size, 20, 20, 256]
                # assert input.get_shape() == [cfg.batch_size, 20, 20, 256]

                # NOTE: I can't find out any words from the paper whether the
                # PrimaryCap convolution does a ReLU activation or not before
                # squashing function, but experiment show that using ReLU get a
                # higher test accuracy. So, which one to use will be your choice
                capsules = tf.contrib.layers.conv2d(input, self.num_outputs * self.vec_len,
                                                    self.kernel_size, self.stride, padding="VALID",
                                                    activation_fn=tf.nn.relu)
                # capsules = tf.contrib.layers.conv2d(input, self.num_outputs * self.vec_len,
                #                                    self.kernel_size, self.stride,padding="VALID",
                #                                    activation_fn=None)
                capsules = tf.reshape(capsules, (cfg.batch_size, -1, self.vec_len, 1))

                # return tensor with shape [batch_size, 1152, 8, 1]
                capsules = squash(capsules)
                return(capsules)

        if self.layer_type == 'FC':
            if self.with_routing:
                # the DigitCaps layer, a fully connected layer
                # Reshape the input into [batch_size, 1152, 1, 8, 1]
                self.input = tf.reshape(input, shape=(cfg.batch_size, -1, 1, input.shape[-2].value, 1))

                with tf.variable_scope('routing'):
                    # b_IJ: [batch_size, num_caps_l, num_caps_l_plus_1, 1, 1],
                    # about the reason of using 'batch_size', see issue #21
                    b_IJ = tf.constant(np.zeros([cfg.batch_size, input.shape[1].value, self.num_outputs, 1, 1], dtype=np.float32))
                    capsules = routing(self.input, b_IJ, num_outputs=self.num_outputs, num_dims=self.vec_len)
                    capsules = tf.squeeze(capsules, axis=1)

            return(capsules)


def routing(input, b_IJ, num_outputs=10, num_dims=16):
    ''' The routing algorithm.

    Args:
        input: A Tensor with [batch_size, num_caps_l=1152, 1, length(u_i)=8, 1]
               shape, num_caps_l meaning the number of capsule in the layer l.
        num_outputs: the number of output capsules.
        num_dims: the number of dimensions for output capsule.
    Returns:
        A Tensor of shape [batch_size, num_caps_l_plus_1, length(v_j)=16, 1]
        representing the vector output `v_j` in the layer l+1
    Notes:
        u_i represents the vector output of capsule i in the layer l, and
        v_j the vector output of capsule j in the layer l+1.
     '''

    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    input_shape = get_shape(input)
    W = tf.get_variable('Weight', shape=[1, input_shape[1], num_dims * num_outputs] + input_shape[-2:],
                        dtype=tf.float32, initializer=tf.random_normal_initializer(stddev=cfg.stddev))
    biases = tf.get_variable('bias', shape=(1, 1, num_outputs, num_dims, 1))

    # Eq.2, calc u_hat
    # Since tf.matmul is a time-consuming op,
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, num_dims * num_outputs, 1, 1])
    # assert input.get_shape() == [cfg.batch_size, 1152, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, input_shape[1], num_outputs, num_dims, 1])
    # assert u_hat.get_shape() == [cfg.batch_size, 1152, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(cfg.iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1152, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == cfg.iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1152, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                # assert s_J.get_shape() == [cfg.batch_size, 1, num_outputs, num_dims, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                # assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < cfg.iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1152, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1152, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, input_shape[1], 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                # assert u_produce_v.get_shape() == [cfg.batch_size, 1152, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v

    return(v_J)


def squash(vector):
    '''Squashing function corresponding to Eq. 1
    Args:
        vector: A tensor with shape [batch_size, 1, num_caps, vec_len, 1] or [batch_size, num_caps, vec_len, 1].
    Returns:
        A tensor with the same shape as vector but squashed in 'vec_len' dimension.
    '''
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)


# capsNet

In [0]:
"""
License: Apache-2.0
Author: Huadong Liao
E-mail: naturomics.liao@gmail.com
"""


epsilon = 1e-9


class CapsNet(object):
    def __init__(self, is_training=True, height=28, width=28, channels=1, num_label=10):
        """
        Args:
            height: Integer, the height of inputs.
            width: Integer, the width of inputs.
            channels: Integer, the channels of inputs.
            num_label: Integer, the category number.
        """
        self.height = height
        self.width = width
        self.channels = channels
        self.num_label = num_label

        self.graph = tf.Graph()

        with self.graph.as_default():
            if is_training:
                self.X, self.labels = get_batch_data(cfg.batch_size, cfg.num_threads)
                self.Y = tf.one_hot(self.labels, depth=self.num_label, axis=1, dtype=tf.float32)

                self.build_arch()
                self.loss()
                self._summary()

                # t_vars = tf.trainable_variables()
                self.global_step = tf.Variable(0, name='global_step', trainable=False)
                self.optimizer = tf.train.AdamOptimizer()
                self.train_op = self.optimizer.minimize(self.total_loss, global_step=self.global_step)
            else:
                self.X = tf.placeholder(tf.float32, shape=(cfg.batch_size, self.height, self.width, self.channels))
                self.labels = tf.placeholder(tf.int32, shape=(cfg.batch_size, ))
                self.Y = tf.reshape(self.labels, shape=(cfg.batch_size, self.num_label, 1))
                self.build_arch()

        tf.logging.info('Seting up the main structure')

    def build_arch(self):
        with tf.variable_scope('Conv1_layer'):
            # Conv1, return tensor with shape [batch_size, 20, 20, 256]
            conv1 = tf.contrib.layers.conv2d(self.X, num_outputs=256,
                                             kernel_size=9, stride=1,
                                             padding='VALID')

        # Primary Capsules layer, return tensor with shape [batch_size, 1152, 8, 1]
        with tf.variable_scope('PrimaryCaps_layer'):
            primaryCaps = CapsLayer(num_outputs=32, vec_len=8, with_routing=False, layer_type='CONV')
            caps1 = primaryCaps(conv1, kernel_size=9, stride=2)

        # DigitCaps layer, return shape [batch_size, 10, 16, 1]
        with tf.variable_scope('DigitCaps_layer'):
            digitCaps = CapsLayer(num_outputs=self.num_label, vec_len=16, with_routing=True, layer_type='FC')
            self.caps2 = digitCaps(caps1)

        # Decoder structure in Fig. 2
        # 1. Do masking, how:
        with tf.variable_scope('Masking'):
            # a). calc ||v_c||, then do softmax(||v_c||)
            # [batch_size, 10, 16, 1] => [batch_size, 10, 1, 1]
            self.v_length = tf.sqrt(reduce_sum(tf.square(self.caps2),
                                               axis=2, keepdims=True) + epsilon)
            self.softmax_v = softmax(self.v_length, axis=1)
            # assert self.softmax_v.get_shape() == [cfg.batch_size, self.num_label, 1, 1]

            # b). pick out the index of max softmax val of the 10 caps
            # [batch_size, 10, 1, 1] => [batch_size] (index)
            self.argmax_idx = tf.to_int32(tf.argmax(self.softmax_v, axis=1))
            # assert self.argmax_idx.get_shape() == [cfg.batch_size, 1, 1]
            self.argmax_idx = tf.reshape(self.argmax_idx, shape=(cfg.batch_size, ))

            # Method 1.
            if not cfg.mask_with_y:
                # c). indexing
                # It's not easy to understand the indexing process with argmax_idx
                # as we are 3-dim animal
                masked_v = []
                for batch_size in range(cfg.batch_size):
                    v = self.caps2[batch_size][self.argmax_idx[batch_size], :]
                    masked_v.append(tf.reshape(v, shape=(1, 1, 16, 1)))

                self.masked_v = tf.concat(masked_v, axis=0)
                assert self.masked_v.get_shape() == [cfg.batch_size, 1, 16, 1]
            # Method 2. masking with true label, default mode
            else:
                self.masked_v = tf.multiply(tf.squeeze(self.caps2), tf.reshape(self.Y, (-1, self.num_label, 1)))
                self.v_length = tf.sqrt(reduce_sum(tf.square(self.caps2), axis=2, keepdims=True) + epsilon)

        # 2. Reconstructe the MNIST images with 3 FC layers
        # [batch_size, 1, 16, 1] => [batch_size, 16] => [batch_size, 512]
        with tf.variable_scope('Decoder'):
            vector_j = tf.reshape(self.masked_v, shape=(cfg.batch_size, -1))
            fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
            fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
            self.decoded = tf.contrib.layers.fully_connected(fc2,
                                                             num_outputs=self.height * self.width * self.channels,
                                                             activation_fn=tf.sigmoid)

    def loss(self):
        # 1. The margin loss

        # [batch_size, 10, 1, 1]
        # max_l = max(0, m_plus-||v_c||)^2
        max_l = tf.square(tf.maximum(0., cfg.m_plus - self.v_length))
        # max_r = max(0, ||v_c||-m_minus)^2
        max_r = tf.square(tf.maximum(0., self.v_length - cfg.m_minus))
        assert max_l.get_shape() == [cfg.batch_size, self.num_label, 1, 1]

        # reshape: [batch_size, 10, 1, 1] => [batch_size, 10]
        max_l = tf.reshape(max_l, shape=(cfg.batch_size, -1))
        max_r = tf.reshape(max_r, shape=(cfg.batch_size, -1))

        # calc T_c: [batch_size, 10]
        # T_c = Y, is my understanding correct? Try it.
        T_c = self.Y
        # [batch_size, 10], element-wise multiply
        L_c = T_c * max_l + cfg.lambda_val * (1 - T_c) * max_r

        self.margin_loss = tf.reduce_mean(tf.reduce_sum(L_c, axis=1))

        # 2. The reconstruction loss
        orgin = tf.reshape(self.X, shape=(cfg.batch_size, -1))
        squared = tf.square(self.decoded - orgin)
        self.reconstruction_err = tf.reduce_mean(squared)

        # 3. Total loss
        # The paper uses sum of squared error as reconstruction error, but we
        # have used reduce_mean in `# 2 The reconstruction loss` to calculate
        # mean squared error. In order to keep in line with the paper,the
        # regularization scale should be 0.0005*784=0.392
        self.total_loss = self.margin_loss + cfg.regularization_scale * self.reconstruction_err

    # Summary
    def _summary(self):
        train_summary = []
        train_summary.append(tf.summary.scalar('train/margin_loss', self.margin_loss))
        train_summary.append(tf.summary.scalar('train/reconstruction_loss', self.reconstruction_err))
        train_summary.append(tf.summary.scalar('train/total_loss', self.total_loss))
        recon_img = tf.reshape(self.decoded, shape=(cfg.batch_size, self.height, self.width, self.channels))
        train_summary.append(tf.summary.image('reconstruction_img', recon_img))
        self.train_summary = tf.summary.merge(train_summary)

        correct_prediction = tf.equal(tf.to_int32(self.labels), self.argmax_idx)
        self.accuracy = tf.reduce_sum(tf.cast(correct_prediction, tf.float32))


# Configs

In [0]:
import tensorflow as tf

flags = tf.app.flags


############################
#    hyper parameters      #
############################

# For separate margin loss
flags.DEFINE_float('m_plus', 0.9, 'the parameter of m plus')
flags.DEFINE_float('m_minus', 0.01, 'the parameter of m minus')
flags.DEFINE_float('lambda_val', 0.5, 'down weight of the loss for absent digit classes')

# for training
flags.DEFINE_integer('batch_size', 128, 'batch size')
flags.DEFINE_integer('epoch', 16, 'epoch')
flags.DEFINE_integer('iter_routing', 3, 'number of iterations in routing algorithm')
flags.DEFINE_boolean('mask_with_y', True, 'use the true label to mask out target capsule or not')

flags.DEFINE_float('stddev', 0.01, 'stddev for W initializer')
## org
#flags.DEFINE_float('regularization_scale', 0.392, 'regularization coefficient for reconstruction loss, default to 0.0005*784=0.392')
flags.DEFINE_float('regularization_scale', 0.392,'modified original 0.392')


############################
#   environment setting    #
############################
flags.DEFINE_boolean('is_training', True, 'train or predict phase')
flags.DEFINE_integer('num_threads', 8, 'number of threads of enqueueing examples')
flags.DEFINE_string('logdir', 'logdir', 'logs directory')
flags.DEFINE_integer('train_sum_freq', 1, 'the frequency of saving train summary(step)')
#flags.DEFINE_integer('val_sum_freq', 429, 'the frequency of saving valuation summary(step)')
flags.DEFINE_integer('save_freq', 1, 'the frequency of saving model(epoch)')
flags.DEFINE_string('results', 'results', 'path for saving results')

# ############################
# #   distributed setting    #
# ############################
# flags.DEFINE_integer('num_gpu', 8, 'number of gpus for distributed training')
# flags.DEFINE_integer('batch_size_per_gpu', 128, 'batch size on 1 gpu')
# flags.DEFINE_integer('thread_per_gpu', 4, 'Number of preprocessing threads per tower.')

cfg = tf.app.flags.FLAGS
tf.logging.set_verbosity(tf.logging.INFO)


# Main Helper functions

In [0]:
def save_to():
    if not os.path.exists(cfg.results):
        os.mkdir(cfg.results)
    if cfg.is_training:
        loss = cfg.results + '/loss.csv'
        train_acc = cfg.results + '/train_acc.csv'
        val_acc = cfg.results + '/val_acc.csv'

        if os.path.exists(val_acc):
            os.remove(val_acc)
        if os.path.exists(loss):
            os.remove(loss)
        if os.path.exists(train_acc):
            os.remove(train_acc)

        fd_train_acc = open(train_acc, 'w')
        fd_train_acc.write('step,train_acc\n')
        fd_loss = open(loss, 'w')
        fd_loss.write('step,loss\n')
        fd_val_acc = open(val_acc, 'w')
        fd_val_acc.write('step,val_acc\n')
        return(fd_train_acc, fd_loss, fd_val_acc)
    else:
        test_acc = cfg.results + '/test_acc.csv'
        if os.path.exists(test_acc):
            os.remove(test_acc)
        fd_test_acc = open(test_acc, 'w')
        fd_test_acc.write('test_acc\n')
        return(fd_test_acc)


def train(model, supervisor, num_label):
    trX, trY, num_tr_batch, valX, valY, num_val_batch = load_data(cfg.batch_size, is_training=True)
    Y = valY[:num_val_batch * cfg.batch_size].reshape((-1, 1))

    fd_train_acc, fd_loss, fd_val_acc = save_to()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with supervisor.managed_session(config=config) as sess:
        print("\nNote: all of results will be saved to directory: " + cfg.results)
        
        best_val_acc = 0
        best_val_loss = 100000000
        for epoch in range(cfg.epoch):
            print("Training for epoch %d/%d:" % (epoch+1, cfg.epoch))
            if supervisor.should_stop():
                print('supervisor stoped!')
                break
            for step in range(num_tr_batch):
                start = step * cfg.batch_size
                end = start + cfg.batch_size
                global_step = epoch * num_tr_batch + step

                if global_step % cfg.train_sum_freq == 0:
                    _, loss, train_acc, summary_str = sess.run([model.train_op, model.total_loss, model.accuracy, model.train_summary])
                    assert not np.isnan(loss), 'Something wrong! loss is nan...'
                    supervisor.summary_writer.add_summary(summary_str, global_step)

                    print("Global step: {}".format(str(global_step)) + ',' + "loss: {}".format(str(loss)) + "\n")


                    fd_loss.write(str(global_step) + ',' + str(loss) + "\n")
                    fd_loss.flush()
                    fd_train_acc.write(str(global_step) + ',' + str(train_acc / cfg.batch_size) + "\n")
                    fd_train_acc.flush()
                else:
                    sess.run(model.train_op)
                
                

                # if cfg.val_sum_freq != 0 and (global_step) % cfg.val_sum_freq == 0:
                #     val_acc = 0
                #     for i in range(num_val_batch):
                #         start = i * cfg.batch_size
                #         end = start + cfg.batch_size
                #         acc = sess.run(model.accuracy, {model.X: valX[start:end], model.labels: valY[start:end]})
                #         val_acc += acc
                #     val_acc = val_acc / (cfg.batch_size * num_val_batch)
                #     print("Global Step: "+str(global_step) + ',' + "val_acc: "+ str(val_acc) + '\n')
                #     fd_val_acc.write(str(global_step) + ',' + str(val_acc) + '\n')
                #     fd_val_acc.flush()

            if (epoch + 1) % cfg.save_freq == 0:
                val_acc = 0
                val_loss =0
                for i in range(num_val_batch):
                    start = i * cfg.batch_size
                    end = start + cfg.batch_size
                    acc, loss = sess.run([model.accuracy,model.total_loss], {model.X: valX[start:end], model.labels: valY[start:end]})
                    val_acc += acc
                    val_loss  += loss 

                val_acc = val_acc / (cfg.batch_size * num_val_batch)
                val_loss = val_loss / (num_val_batch)

                # if ((val_loss < best_val_loss) and (best_val_acc < val_acc) ):
                if ((val_loss < best_val_loss) ):
                  best_val_loss = val_loss
                  # best_val_acc = val_acc
                  print("\n##################### Saving Model ############################\n")
                  print("Global Step: "+str(global_step) + ',' + "Val_Loss: "+ str(val_loss)+ ", " +" Val_acc: "+ str(val_acc) + " Improved"+'\n')
                  fd_val_acc.write(str(global_step) + ',' + str(val_acc) + '\n')
                  fd_val_acc.flush()
                  supervisor.saver.save(sess, cfg.logdir + '/model_epoch_%04d_step_%02d' % (epoch, global_step))
                else:
                  print("\n######NOT SAVING MODEL #########\n")
                  print("Global Step: "+str(global_step) + ',' + "val_loss: "+ str(val_loss) +'\n')

        fd_val_acc.close()
        fd_train_acc.close()
        fd_loss.close()


def evaluation(model, supervisor, num_label):
    teX, teY, num_te_batch = load_data(cfg.batch_size, is_training=False)
    fd_test_acc = save_to()
    with supervisor.managed_session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
        supervisor.saver.restore(sess, tf.train.latest_checkpoint(cfg.logdir))
        tf.logging.info('Model restored!')

        test_acc = 0
        for i in range(num_te_batch):
            start = i * cfg.batch_size
            end = start + cfg.batch_size
            acc = sess.run(model.accuracy, {model.X: teX[start:end], model.labels: teY[start:end]})
            test_acc += acc
        test_acc = test_acc / (cfg.batch_size * num_te_batch)
        fd_test_acc.write(str(test_acc))
        fd_test_acc.close()
        print('\nTest Accuracy is {}:'.format(test_acc))
        print('\nTest accuracy has been saved to ' + cfg.results + '/test_acc')
        




# Main function

In [0]:
cfg.is_training=True
# try:
def main(_):
    tf.logging.info(' Loading Graph...')
    num_label = 10
    model = CapsNet()
    tf.logging.info(' Graph loaded')
    tf.logging.set_verbosity(tf.logging.INFO)


    sv = tf.train.Supervisor(graph=model.graph, logdir=cfg.logdir, save_model_secs=0)

    if cfg.is_training:
        tf.logging.info(' Start training...')
        train(model, sv, num_label)
        tf.logging.info('Training done')
    else:
        evaluation(model, sv, num_label)

if __name__ == "__main__":
    tf.app.run()

# except:
#   print("\nBeginning Eval")

INFO:tensorflow: Loading Graph...


I1112 10:03:57.781072 139738004481920 <ipython-input-10-fd696e1d0d24>:4]  Loading Graph...



Starting Data Augmentation

Finished Augmentation
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


W1112 10:04:05.403683 139738004481920 deprecation.py:323] From <ipython-input-5-c01d0c89e12e>:56: slice_input_producer (from tensorflow.python.training.input) is deprecated and will be removed in a future version.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.


...

I1112 10:18:52.492827 139735703181056 supervisor.py:1099] global_step/sec: 6.67135


Global step: 5582,loss: 0.009118574

Global step: 5583,loss: 0.008708395

Global step: 5584,loss: 0.011854198

Global step: 5585,loss: 0.008749703

Global step: 5586,loss: 0.009568879

Global step: 5587,loss: 0.008775931

Global step: 5588,loss: 0.011785828

Global step: 5589,loss: 0.008281715

Global step: 5590,loss: 0.0071160235

Global step: 5591,loss: 0.008524633

Global step: 5592,loss: 0.008145527

Global step: 5593,loss: 0.0075587137

Global step: 5594,loss: 0.009676316

Global step: 5595,loss: 0.010142857

Global step: 5596,loss: 0.011124147

Global step: 5597,loss: 0.008739995

Global step: 5598,loss: 0.008347003

Global step: 5599,loss: 0.007384679

Global step: 5600,loss: 0.013360244

Global step: 5601,loss: 0.011811178

Global step: 5602,loss: 0.009398264

Global step: 5603,loss: 0.007649317

Global step: 5604,loss: 0.009141941

Global step: 5605,loss: 0.011778055

Global step: 5606,loss: 0.010227515

Global step: 5607,loss: 0.0096826255

Global step: 5608,loss: 0.016344994

I1112 10:20:33.682474 139738004481920 <ipython-input-10-fd696e1d0d24>:16] Training done


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
cfg.is_training=False

try:
  def main(_):
      tf.logging.info(' Loading Graph...')
      num_label = 10
      model = CapsNet()
      tf.logging.info(' Graph loaded')
      tf.logging.set_verbosity(tf.logging.INFO)


      sv = tf.train.Supervisor(graph=model.graph, logdir=cfg.logdir, save_model_secs=0)

      if cfg.is_training:
          tf.logging.info(' Start training...')
          train(model, sv, num_label)
          tf.logging.info('Training done')
      else:
          evaluation(model, sv, num_label)

  if __name__ == "__main__":
      tf.app.run()

except:
  print("\Completed")

INFO:tensorflow: Loading Graph...


I1112 10:24:52.461792 139738004481920 <ipython-input-12-d583bcff4fbf>:5]  Loading Graph...



Starting Data Augmentation

Finished Augmentation
INFO:tensorflow:Seting up the main structure


I1112 10:25:01.494881 139738004481920 <ipython-input-7-4752a8a30df1>:46] Seting up the main structure


INFO:tensorflow: Graph loaded


...

INFO:tensorflow:Running local_init_op.


I1112 10:25:02.361317 139738004481920 session_manager.py:500] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I1112 10:25:02.392027 139738004481920 session_manager.py:502] Done running local_init_op.


\Completed
